<a href="https://colab.research.google.com/github/sketcher03/learning_app_v1/blob/main/sentence2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame with specified encoding
df = pd.read_csv('/content/filtered_dataset.csv', encoding='ISO-8859-1')

# Display the first few rows
print(df.head(10))



    sentences words
0   Am I fat?   fat
1   Be a man.   man
2   I did OK.   did
3   I did it.   did
4   I get it.   get
5   I got it.   got
6   I saw it.   saw
7   I use it.   use
8   May I go?    go
9  Am I late?  late


In [ ]:
import pandas as pd
from google.colab import files

# Upload the Excel file
uploaded = files.upload()

# Load the Excel file into a pandas DataFrame
df = pd.read_csv(next(iter(uploaded)))

# Display the first few rows
print(df.head(10))

Saving filtered_dataset.csv to filtered_dataset (1).csv
    sentences words
0   Am I fat?   fat
1   Be a man.   man
2   I did OK.   did
3   I did it.   did
4   I get it.   get
5   I got it.   got
6   I saw it.   saw
7   I use it.   use
8   May I go?    go
9  Am I late?  late


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import torch


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq

# Load the GPT-2 tokenizer and set pad_token to eos_token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

# Function to tokenize data
def tokenize_data(row):
    input_text = row['words']
    output_text = row['sentences']

    # Tokenize the input and output with padding
    inputs = tokenizer(input_text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    outputs = tokenizer(output_text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)

    # Return input_ids and labels (output sequence as labels)
    return {
        'input_ids': inputs['input_ids'].squeeze(0).tolist(),
        'labels': outputs['input_ids'].squeeze(0).tolist()
    }

# Apply the tokenization function to your dataset
tokenized_data = df.apply(tokenize_data, axis=1)

# Convert tokenized data into Hugging Face Dataset format
train_data = [{'input_ids': tokenized_data[i]['input_ids'], 'labels': tokenized_data[i]['labels']} for i in range(len(tokenized_data))]
dataset = Dataset.from_list(train_data)

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define the DataCollator for padding sequences
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Directory to save the model
    run_name="CSE",                    # Experiment name
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=1,     # Batch size per device
    logging_dir='./logs',              # Log directory
    save_steps=500,                    # Save checkpoints every 500 steps
    evaluation_strategy="no",          # Evaluation strategy
)

# Initialize Trainer
trainer = Trainer(
    model=model,                      # Model to train
    args=training_args,               # Training arguments
    data_collator=data_collator,      # Data collator for padding
    train_dataset=dataset             # Tokenized dataset
)

# Train the model
trainer.train()


In [ ]:
# Function to generate sentences from a word
def generate_sentence(word, model, tokenizer, max_length=50):
    input_ids = tokenizer.encode(word, return_tensors="pt")  # Convert word to token IDs
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example test
generated_sentence = generate_sentence("happy", model, tokenizer)
print("Generated Sentence:", generated_sentence)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
import pandas as pd

# Load the GPT-2 tokenizer and set pad_token to eos_token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

# Function to tokenize data
def tokenize_data(row):
    input_text = str(row['words']) if pd.notna(row['words']) else ""
    output_text = str(row['sentences']) if pd.notna(row['sentences']) else ""

    # Tokenize the input and output with padding
    inputs = tokenizer(input_text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    outputs = tokenizer(output_text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)

    # Return input_ids and labels (output sequence as labels)
    return {
        'input_ids': inputs['input_ids'].squeeze(0).tolist(),
        'labels': outputs['input_ids'].squeeze(0).tolist()
    }

# Apply the tokenization function to your dataset
tokenized_data = df.apply(tokenize_data, axis=1)

# Convert tokenized data into Hugging Face Dataset format
train_data = [{'input_ids': tokenized_data[i]['input_ids'], 'labels': tokenized_data[i]['labels']} for i in range(len(tokenized_data))]
dataset = Dataset.from_list(train_data)

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define the DataCollator for padding sequences
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Directory to save the model
    run_name="CSE",                    # Experiment name
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=2,     # Batch size per device
    logging_dir='./logs',              # Log directory
    save_steps=500,                    # Save checkpoints every 500 steps
    evaluation_strategy="no",          # Evaluation strategy
)

# Initialize Trainer
trainer = Trainer(
    model=model,                      # Model to train
    args=training_args,               # Training arguments
    data_collator=data_collator,      # Data collator for padding
    train_dataset=dataset             # Tokenized dataset
)

# Train the model
trainer.train()


In [ ]:
#for_fast_training
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
import pandas as pd

# Load the smaller, faster GPT-2 model (DistilGPT-2)
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load the model and enable gradient checkpointing
model = GPT2LMHeadModel.from_pretrained(model_name)
model.gradient_checkpointing_enable()  # Saves memory

# Efficient batch tokenization function
def tokenize_data(examples):
    input_texts = [str(word) for word in examples['words']]
    output_texts = [str(sentence) for sentence in examples['sentences']]

    inputs = tokenizer(input_texts, padding="max_length", max_length=512, truncation=True)
    outputs = tokenizer(output_texts, padding="max_length", max_length=512, truncation=True)

    return {'input_ids': inputs['input_ids'], 'labels': outputs['input_ids']}

# Convert dataframe to Hugging Face dataset and tokenize efficiently
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_data, batched=True, remove_columns=['words', 'sentences'])

# Use optimized data collator for GPT-2
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 does not use masked language modeling
)

# Define optimized training arguments
training_args = TrainingArguments(
    output_dir='./results',
    run_name="CSE",
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Increased batch size
    gradient_accumulation_steps=2,  # Helps with larger batch sizes
    logging_steps=1000,  # Log less frequently
    save_steps=5000,  # Save checkpoints less frequently
    evaluation_strategy="no",
    fp16=True,  # Mixed precision for faster training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anjum-rifah (anjum-rifah-north-south-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
import pandas as pd
import torch

# Load model and tokenizer (smaller & faster version)
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load model and enable optimizations
model = GPT2LMHeadModel.from_pretrained(model_name)
model.gradient_checkpointing_enable()  # Reduce memory
if torch.__version__ >= "2.0":
    model = torch.compile(model)  # Speed up execution

# Efficient batch tokenization
def tokenize_data(examples):
    input_texts = [str(word) for word in examples['words']]
    output_texts = [str(sentence) for sentence in examples['sentences']]
    inputs = tokenizer(input_texts, padding="max_length", max_length=256, truncation=True)
    outputs = tokenizer(output_texts, padding="max_length", max_length=256, truncation=True)
    return {'input_ids': inputs['input_ids'], 'labels': outputs['input_ids']}

# Convert dataset and tokenize efficiently
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_data, batched=True, remove_columns=['words', 'sentences'])
dataset = dataset.select(range(10000))  # Reduce dataset size for testing

# Optimized Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Optimized Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,  # Increase batch size
    gradient_accumulation_steps=1,
    logging_steps=2000,  # Log less frequently
    save_strategy="epoch",  # Save only at the end of epochs
    fp16=True if not torch.cuda.is_bf16_supported() else False,
    bf16=torch.cuda.is_bf16_supported(),
    dataloader_num_workers=4,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train
trainer.train()


Map:   0%|          | 0/120152 [00:00<?, ? examples/s]

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [labels, input_ids]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
print(df.head())  # Print first few rows


   sentences words
0  Am I fat?   fat
1  Be a man.   man
2  I did OK.   did
3  I did it.   did
4  I get it.   get


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
import pandas as pd
import torch

# Load model and tokenizer (smaller & faster version)
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load GPT-2 model with optimizations
model = GPT2LMHeadModel.from_pretrained(model_name)
model.gradient_checkpointing_enable()  # Reduce memory usage
if torch.__version__ >= "2.0":
    model = torch.compile(model)  # Speed up execution

# Load dataset (Ensure df is defined)
df = pd.read_csv("filtered_dataset.csv")  # Change to your actual dataset file

# Ensure dataset has correct columns
if "words" not in df.columns or "sentences" not in df.columns:
    raise ValueError("Dataset must contain 'words' and 'sentences' columns!")

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Efficient batch tokenization
def tokenize_data(examples):
    inputs = tokenizer(examples["words"], padding="max_length", max_length=10, truncation=True)
    outputs = tokenizer(examples["sentences"], padding="max_length", max_length=50, truncation=True)

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": outputs["input_ids"]
    }

# Apply tokenization
dataset = dataset.map(tokenize_data, batched=True, remove_columns=["words", "sentences"])

# Reduce dataset size for faster testing (Optional)
dataset = dataset.select(range(min(len(dataset), 10000)))

# Optimized Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Optimized Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Adjust based on GPU memory
    gradient_accumulation_steps=2,
    logging_steps=1000,  # Log less frequently
    save_strategy="epoch",  # Save only at the end of epochs
    fp16=torch.cuda.is_available() and not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    dataloader_num_workers=4,
    remove_unused_columns=False,  # Fix for missing dataset columns
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train the model
trainer.train()


Map:   0%|          | 0/120152 [00:00<?, ? examples/s]

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.